# Carteira de Dividendos
> **Minimização dos Riscos**

- Universidade Federal da Paraíba
- Ciência de Dados para Negócios
- Disciplina: Otimização Aplicada A Negócios
- Docente: Jorge Henrique Norões Viana
- Discentes: Lucas Rabay, Nercino Neto e Pedro Henrique

## **Instalação dos pacotes**

In [ ]:
!apt-get install -y -qq glpk-utils
!pip install -q pyomo

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

#### **Desativando avisos**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

___

## **Dados**

Nessa seção, os dados necessários para a modelagem serão obtidos e tratados.{

In [ ]:
# Import module
from pandas import read_excel

# Data
df_dividendos = read_excel('/content/Dividendos - MAD.xlsx', names=['asset','dy_mad'])
df_dividendos_media = read_excel('/content/Dividendos - Média - 5 Anos.xlsx', names=['asset','dy_medio'])
df_precos = read_excel('/content/Preços - MAD.xlsx', names=['asset','pr_medio'])

In [ ]:
# Replacing wrong 'BBAS3' for 'BBSA3'
df_dividendos['asset'] = df_dividendos['asset'].replace('BBAS3', 'BBSA3')
df_dividendos_media['asset'] = df_dividendos_media['asset'].replace('BBAS3', 'BBSA3')
df_precos['asset'] = df_precos['asset'].replace('BBAS3', 'BBSA3')

In [ ]:
# Joining all the data
df = (
    df_dividendos.join(
        df_dividendos_media.set_index('asset'),
        on='asset',
        how='inner'
    ).join(
        df_precos.set_index('asset'),
        on='asset',
        how='inner'
    )
)

In [ ]:
# Result
df

,asset,dy_mad,dy_medio,pr_medio
0,TAEE11,0.021900,0.099173,0.090803
1,VIVT3,0.016763,0.060711,0.162668
2,BBSE3,0.021233,0.070420,0.256544
3,SANB11,0.015074,0.064221,0.108149
4,EGIE3,0.014583,0.053694,0.116560
5,ITSA4,0.011850,0.063601,0.156251
6,BBSA3,0.022706,0.076782,0.305559
7,BRAP4,0.098700,0.142050,0.261964
8,CMIG4,0.038250,0.099123,0.086668
9,CPLE6,0.054330,0.073391,0.165563


### **Pesos**

Nessa seção, os pesos para cada ativo será extraído a partir do site oficial da B3.

URL: 'https://www.b3.com.br/pt_br/noticias/b3-lanca-primeiro-indice-derivado-do-ibovespa-b3-para-acompanhar-empresas-que-mais-pagam-dividendos.htm'


In [ ]:
from bs4 import BeautifulSoup
from requests import get as request_get

import numpy as np
import pandas as pd

# Site URL
url = 'https://www.b3.com.br/pt_br/noticias/b3-lanca-primeiro-indice-derivado-do-ibovespa-b3-para-acompanhar-empresas-que-mais-pagam-dividendos.htm'

# HTML Parser
html = request_get(url).text
soup = BeautifulSoup(html, 'html.parser')

# Finding weghts table
table = soup.find('table')

# Reading and cleaning
df_weights = pd.read_html(str(table))[0]
df_weights.columns = ['asset','company','weight']
df_weights['weight'] = df_weights['weight'].str.replace('%','').astype(float).apply(lambda weight: weight/100)

# Join with main DataFrane
df = df.join(
    df_weights.set_index('asset'),
    on='asset',
    how='inner'
)

# Reordering columns
df = df[['asset','dy_mad','dy_medio','pr_medio','weight','company']]
df

,asset,dy_mad,dy_medio,pr_medio,weight,company
0,TAEE11,0.021900,0.099173,0.090803,0.0580,Taesa
1,VIVT3,0.016763,0.060711,0.162668,0.0580,Telefônica Brasil
2,BBSE3,0.021233,0.070420,0.256544,0.0569,BB Seguridade
3,SANB11,0.015074,0.064221,0.108149,0.0554,Banco Santander (Brasil)
4,EGIE3,0.014583,0.053694,0.116560,0.0549,Engie Brasil
5,ITSA4,0.011850,0.063601,0.156251,0.0536,Itausa AS
6,BBSA3,0.022706,0.076782,0.305559,0.0531,Banco do Brasil
7,BRAP4,0.098700,0.142050,0.261964,0.0531,Bradespar
8,CMIG4,0.038250,0.099123,0.086668,0.0531,Companhia Energética de Minas Gerais S.A
9,CPLE6,0.054330,0.073391,0.165563,0.0483,Copel


### **Vetores**

In [ ]:
# Creating vars vectors
weight_vectors = np.array(df['weight'])
dy_mad_vectors = np.array(df['dy_mad'])
dy_medio_vectors = np.array(df['dy_medio'])
pr_medio_vectors = np.array(df['pr_medio'])

print(f'weight_vectors: {weight_vectors}\n')
print(f'dy_mad_vectors: {dy_mad_vectors}\n')
print(f'dy_medio_vectors: {dy_medio_vectors}\n')
print(f'pr_medio_vectors: {pr_medio_vectors}\n')

weight_vectors: [0.058  0.058  0.0569 0.0554 0.0549 0.0536 0.0531 0.0531 0.0531 0.0483
 0.0472 0.0462 0.0459 0.0452 0.0439 0.0438 0.0424 0.0421 0.0413 0.028 ]

dy_mad_vectors: [0.0219     0.01676292 0.02123333 0.01507355 0.01458333 0.01185
 0.02270616 0.09869988 0.03825    0.05433026 0.02938333 0.04263333
 0.04278605 0.0374     0.0208     0.01176667 0.05671667 0.01415
 0.14666667 0.08470284]

dy_medio_vectors: [0.09917346 0.06071123 0.07042023 0.06422064 0.05369418 0.06360062
 0.07678152 0.14204963 0.09912256 0.07339078 0.07660698 0.08945154
 0.08555816 0.06306262 0.0652994  0.05209458 0.09750681 0.0519166
 0.19525742 0.06030853]

pr_medio_vectors: [0.09080335 0.16266779 0.25654375 0.10814942 0.11655994 0.15625102
 0.3055595  0.26196351 0.08666819 0.16556324 0.14858124 0.49910158
 0.24833479 0.1604709  0.21526724 0.31170796 0.13027111 0.52831199
 0.27724938 0.36030059]



In [ ]:
# Creating vars indexes
dy_mad_indexes = float(sum(weight_vectors * dy_mad_vectors))
dy_medio_indexes = float(sum(weight_vectors * dy_medio_vectors))
pr_medio_indexes = float(sum(weight_vectors * pr_medio_vectors))

In [ ]:
# Creating a dict for mappin all tickers and your respectives vars
returns = {ticker:dy for ticker, dy in zip(df['asset'], df['dy_medio'])}
dy_mad_dict = {ticker:dy for ticker, dy in zip(df['asset'], df['dy_mad'])}
pr_medio_dict = {ticker:dy for ticker, dy in zip(df['asset'], df['pr_medio'])}

## **Modelagem**

In [ ]:
# Import Modules
from pyomo.environ import *
# Solver
solver = SolverFactory('glpk')

## **Modelo**

$
\max_{w_i} \sum_i w_i r_i
$
<br>
<br>
$
\text{Sujeito à:}
$

$
w_i \geq 0
$

$
\sum_i w_i = 1 \quad \text{(soma dos pesos)}
$

$
w_i \leq 0,20 \quad \text{(concentração individual)}
$

$
\sum_{i \in EN} w_i \leq 0,50 \quad \text{(concentração energia)}
$

$
\sum_{i \in FI} w_i \leq 0,50 \quad \text{(concentração financeiro)}
$

$
\sum_{i \in DE} w_i \leq 0,50 \quad \text{(concentração demais)}
$

$
\sum_i w_i \, mad_{i,dy} \leq mad_{ref,dy}
$
$
\sum_i w_i \, mad_{i,pr} \leq mad_{ref,pr}
$

In [ ]:
# Tickets
FI = ['BBSE3', 'SANB11', 'ITSA4', 'BBSA3', 'BRAP4', 'CIEL3']
EN = ['TAEE11', 'EGIE3', 'CMIG4', 'CPLE6', 'CPFE3'];
DE = ['VIVT3', 'CSNA3', 'VALE3', 'GGBR4', 'VBBR3', 'CYRE3', 'GOAU4', 'PETR4', 'MRFG3']

assets = list(df['asset'])

In [ ]:
class MODEL:
  """Model generation class"""
  def __init__(self, scenario, assets, FI, EN, DE) -> None:
    """Model generation constructor"""
    self.scenario = scenario
    self.assets = assets
    self.FI = FI
    self.EN = EN
    self.DE = DE

  def gen_scenarios_data(self) -> tuple:
    """Scenario data generator"""

    if self.scenario == 'optimistic':
      # Optimistic scenario adjustments
      max_individual = 0.25
      max_sector = 0.50
      dividend_adjustment = 1.10
      mad_dividend_adjustment = 0.90
      mad_price_adjustment = 0.80

    elif self.scenario == 'pessimistic':
      # Pessimistic scenario adjustments
      max_individual = 0.15
      max_sector = 0.40
      dividend_adjustment = 0.90
      mad_dividend_adjustment = 1.10
      mad_price_adjustment = 1.20

    else:
      # Default scenario adjustments
      max_individual = 0.20
      max_sector = 0.50
      dividend_adjustment = 1.00  # No dividend adjustment
      mad_dividend_adjustment = 1.00  # No dividend adjustment
      mad_price_adjustment = 1.00  # No dividend adjustment

    return (
        max_individual,
        max_sector,
        dividend_adjustment,
        mad_dividend_adjustment,
        mad_price_adjustment
            )

  def gen_model(self) -> object:
    """Model generator"""
    # Model
    MODEL = ConcreteModel()
    # Assets
    MODEL.assets = Set(initialize=self.assets)
    # Decision var
    MODEL.w = Var(MODEL.assets, domain=NonNegativeReals, bounds=(0, 1))

    max_individual, max_sector, dividend_adjustment, mad_dividend_adjustment, mad_price_adjustment = self.gen_scenarios_data()

    # Objective (minimize risks)
    objective = lambda model: (sum((dy_mad_dict[i] * mad_dividend_adjustment) * model.w[i] + (pr_medio_dict[i] * mad_price_adjustment) * model.w[i] for i in model.assets))
    MODEL.obj = Objective(rule=objective, sense=minimize)

    # Assets sets
    MODEL.FI = Set(initialize=self.FI)
    MODEL.EN = Set(initialize=self.EN)
    MODEL.DE = Set(initialize=self.DE)

    return MODEL

  def gen_constraints(self) -> object:
    """Constraints generator"""
    # Vars
    max_individual, max_sector, dividend_adjustment, mad_dividend_adjustment, mad_price_adjustment = self.gen_scenarios_data()

    # Model
    MODEL = self.gen_model()

    # Individual Concentrations
    MODEL.individual_limit = Constraint(MODEL.assets, rule=lambda model, i: model.w[i] <= max_individual)
    # Sectors Restrictions
    MODEL.sector_limit_fi = Constraint(expr=sum(MODEL.w[i] for i in MODEL.FI) <= max_sector)
    MODEL.sector_limit_en = Constraint(expr=sum(MODEL.w[i] for i in MODEL.EN) <= max_sector)
    MODEL.sector_limit_de = Constraint(expr=sum(MODEL.w[i] for i in MODEL.DE) <= max_sector)
    # Total Constraint
    MODEL.sum_w = Constraint(expr=sum(MODEL.w[i] for i in MODEL.assets) == 1)

    return MODEL

In [ ]:
model_BAU = MODEL('BAU', assets, FI, EN, DE).gen_constraints()
model_BAU.dual = Suffix(direction=Suffix.IMPORT)
model_optimistic = MODEL('optimistic', assets, FI, EN, DE).gen_constraints()
model_pessimistic = MODEL('pessimistic', assets, FI, EN, DE).gen_constraints()

### Testando modelo base:

In [ ]:
results_BAU = solver.solve(model_BAU)

print(f'Status: {results_BAU.solver.status}')
print(f'Termination condition: {results_BAU.solver.termination_condition}')

Status: ok
Termination condition: optimal


In [ ]:
model_BAU.pprint()

4 Set Declarations
    DE : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {'VIVT3', 'CSNA3', 'VALE3', 'GGBR4', 'VBBR3', 'CYRE3', 'GOAU4', 'PETR4', 'MRFG3'}
    EN : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'TAEE11', 'EGIE3', 'CMIG4', 'CPLE6', 'CPFE3'}
    FI : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'BBSE3', 'SANB11', 'ITSA4', 'BBSA3', 'BRAP4', 'CIEL3'}
    assets : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {'TAEE11', 'VIVT3', 'BBSE3', 'SANB11', 'EGIE3', 'ITSA4', 'BBSA3', 'BRAP4', 'CMIG4', 'CPLE6', 'CPFE3', 'CSNA3', 'VALE3', 'GGBR4', 'VBBR3', 'CYRE3', 'GOAU4', 'CIEL3', 'PETR4', 'MRFG3'}

1 Var Declarations
    w : Size=20, Index=assets
        Key    : Lower : Va

In [ ]:
model_BAU.obj()

0.13684650395653902

In [ ]:
def calculate_returns(model):
  weights = {ticker: model.w[ticker].value for ticker in model.assets}
  return sum(returns[ticker] * weights[ticker] for ticker in model.assets)

In [ ]:
calculate_returns(model_BAU)

0.07666399749548633

**Composição da Carteira Ótima (cenário básico):**

> Adicionar aspas



1. Setor Energia - 50%

- CMIG4 - 20%
- TAEE11 - 20%
- EGIE3 - 10%

2. Setor Financeiro - 40%

- ITSA4 - 20%
- SANB11 - 20%

3. Demais Setores - 10%

- VIVT3 - 10%

Essa Carteira Possui um MAD total de: 0.13684650395653902 e um retorno ótimo de 7,67%.

Nesse caso, podemos observar que a carteira otimizada, minimizando MAD obtém um retorno inferior ao índice (7,67% x 8,17%), mas alcança uma volatilidade significativamente inferior(0.1368 x 0.2585). Dessa maneira, a carteira apresenta-se como uma excelente alternativa para investidores mais conservadores.

**Preços Sombra**

Os preços sombra encontrado indicam o seguinte:

- **individual_limit[CMIG4]** : -0.00622507941953021 - indica que permitir um aumento a concentração de CMIG4 na carteira reduziria o MAD.
- **individual_limit[ITSA4]** :  -0.0113296953148142 - indica, assim como no caso anterior, que se relaxada a restrição de concentração individual, um aumento em ITSA4 reduziria o MAD, nesse caso de maneira ainda mais significativa.
- **individual_limit[SANB11]**:  -0.0562077504168538 e **individual_limit[TAEE11]** :  -0.0184399201580793 - segue a mesma lógica das duas primeiras.
- **sector_limit_en** :  -0.0482874393680861 - indica que permitir um aumento na concentração setorial permitiria reduzir o MAD.

sum_w :    0.179430712702438 - é irreal e indica que ter mais de 100% da carteira aumentaria o MAD.

In [ ]:
results_optimistic = solver.solve(model_optimistic)

print(f'Status: {results_optimistic.solver.status}')
print(f'Termination condition: {results_optimistic.solver.termination_condition}')

Status: ok
Termination condition: optimal


In [ ]:
model_optimistic.pprint()

4 Set Declarations
    DE : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {'VIVT3', 'CSNA3', 'VALE3', 'GGBR4', 'VBBR3', 'CYRE3', 'GOAU4', 'PETR4', 'MRFG3'}
    EN : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'TAEE11', 'EGIE3', 'CMIG4', 'CPLE6', 'CPFE3'}
    FI : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'BBSE3', 'SANB11', 'ITSA4', 'BBSA3', 'BRAP4', 'CIEL3'}
    assets : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {'TAEE11', 'VIVT3', 'BBSE3', 'SANB11', 'EGIE3', 'ITSA4', 'BBSA3', 'BRAP4', 'CMIG4', 'CPLE6', 'CPFE3', 'CSNA3', 'VALE3', 'GGBR4', 'VBBR3', 'CYRE3', 'GOAU4', 'CIEL3', 'PETR4', 'MRFG3'}

1 Var Declarations
    w : Size=20, Index=assets
        Key    : Lower : Va

In [ ]:
model_optimistic.obj()

0.10796594403439529

In [ ]:
calculate_returns(model_optimistic)

0.08152932111405725

**Composição da Carteira Ótima (cenário otimista):**

1. Setor Energia - 50%

- CMIG4 - 25%
- TAEE11 - 25%

2. Setor Financeiro - 50%

- ITSA4 - 25%
- SANB11 - 25%

Essa Carteira Possui um MAD total de: 0.10796594403439529 e um retorno ótimo de 8,15%.

No cenário otimista a carteira torna-se mais condesada em certos ativos, eliminando os que estava em concentração de 10% na carteira do cenário básico e incorporando esses 20% "a mais", nas carteiras que eram dominantes.

Observamos que o cenário otimista é melhor do que o básico em ambos os fatores. Com um MAD ligeiramente menor (0.1080 x 0.1368), e um retorno superior (8,15% x 7.67%). Inclusive em comparação com o indíce, seu retorna se põe extremamente próximo, com menos da metade da volatilidade.

In [ ]:
results_pessimistic = solver.solve(model_pessimistic)

print(f'Status: {results_pessimistic.solver.status}')
print(f'Termination condition: {results_pessimistic.solver.termination_condition}')

Status: ok
Termination condition: optimal


In [ ]:
model_pessimistic.pprint()

4 Set Declarations
    DE : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {'VIVT3', 'CSNA3', 'VALE3', 'GGBR4', 'VBBR3', 'CYRE3', 'GOAU4', 'PETR4', 'MRFG3'}
    EN : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'TAEE11', 'EGIE3', 'CMIG4', 'CPLE6', 'CPFE3'}
    FI : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'BBSE3', 'SANB11', 'ITSA4', 'BBSA3', 'BRAP4', 'CIEL3'}
    assets : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {'TAEE11', 'VIVT3', 'BBSE3', 'SANB11', 'EGIE3', 'ITSA4', 'BBSA3', 'BRAP4', 'CMIG4', 'CPLE6', 'CPFE3', 'CSNA3', 'VALE3', 'GGBR4', 'VBBR3', 'CYRE3', 'GOAU4', 'CIEL3', 'PETR4', 'MRFG3'}

1 Var Declarations
    w : Size=20, Index=assets
        Key    : Lower : Va

In [ ]:
model_pessimistic.obj()

0.17434858539550413

In [ ]:
calculate_returns(model_pessimistic)

0.07801971552511668

**Composição da Carteira Ótima (cenário pessimista):**

1. Setor Energia - 40%

- CMIG4 - 15%
- TAEE11 - 15%
- EGIE3 - 10%

2. Setor Financeiro - 30%

- ITSA4 - 15%
- SANB11 - 15%

3. Demais Setores - 30%

- GOAU4 - 15%
- VIVT3 - 15%

Essa Carteira Possui um MAD total de: 0.17434858539550413 e um retorno ótimo de 7,80%.

No cenário pessimista, dadas a restrições mais rígidas de concentração, a carteira se diversifica mais, inclusive adicionando um novo ativo **(GOAU4)**.

Ademais, observa-se que interessantemente nesse cenário, apesar de obter uma volatilidade superior ao cenário básico (0.1743 x 0.1368), obteve um rendimento superior (7,80% x 7.67%), possivelmente as mudanças no MAD dos ativos, reduziram o 'benefício' de escolher ativos originalmente mais seguros e acabaram aumentando a renatbilidade.